In [2]:
import numpy as np, pandas as pd
from pulp import *
from ortoolpy import addvars, addbinvars
from IPython.core.display import display

In [3]:
a = pd.read_html('/tmp/shouta.html')[1].T

In [4]:
a,a.columns = a.iloc[1:],a.iloc[0].tolist()
a.必要人数 = a.必要人数.astype(int)
a.iloc[:,2:] = ~a.iloc[:,2:].isnull()
a.insert(0, '曜日', a.index.str[0])
a.reset_index(drop=True, inplace=True)
a = a.iloc[:,list(range(3,a.shape[1]))+[0,1,2]]
display(a[:3]) # 最初の3行表示


,従業員0,従業員1,従業員2,従業員3,従業員4,従業員5,従業員6,従業員7,従業員8,従業員9,曜日,時間帯,必要人数
0,True,True,False,True,False,True,False,False,False,False,月,朝,2
1,False,True,False,True,False,True,False,True,False,False,月,昼,3
2,False,True,False,True,True,True,False,False,True,False,月,夜,3


In [5]:
# Nコマには21、N従業員には10と長さを取っているだけのようだ
# NはNumberの頭文字と思われる
Nコマ,N従業員 = a.shape[0],a.shape[1]-3
display(Nコマ)
display(N従業員)

21

10

In [6]:
# L従業員は単に従業員を列挙したもの
# LはListの頭文字と思われる
L従業員 = list(range(N従業員))
display(L従業員)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [7]:
L管理者 = [3,5,9] # 管理者は従業員3, 5, 9
C必要人数差 = 10
C希望不可 = 100
C最低コマ数 = 1
C管理者不足 = 100
C1日2コマ = 10

In [8]:
m = LpProblem() # 数理モデル

In [9]:
# V割当はコマを示しているようだ
# 21* 10 = 210コマ
V割当 = np.array(addbinvars(Nコマ, N従業員))

In [10]:
V最低コマ数 = addvars(N従業員)
display(V最低コマ数)

[v0211, v0212, v0213, v0214, v0215, v0216, v0217, v0218, v0219, v0220]

In [11]:
a['V必要人数差'] = addvars(Nコマ)
a['V管理者不足'] = addvars(Nコマ)
display(a[:3])

,従業員0,従業員1,従業員2,従業員3,従業員4,従業員5,従業員6,従業員7,従業員8,従業員9,曜日,時間帯,必要人数,V必要人数差,V管理者不足
0,True,True,False,True,False,True,False,False,False,False,月,朝,2,v0221,v0242
1,False,True,False,True,False,True,False,True,False,False,月,昼,3,v0222,v0243
2,False,True,False,True,True,True,False,False,True,False,月,夜,3,v0223,v0244


In [12]:
V1日2コマ = addvars(N従業員)
display(V1日2コマ)

[v0263, v0264, v0265, v0266, v0267, v0268, v0269, v0270, v0271, v0272]

In [13]:
m += (C必要人数差 * lpSum(a.V必要人数差)
    + C希望不可 * lpSum(a.apply(lambda r: lpDot(1-r[L従業員],V割当[r.name]), 1))
    + C最低コマ数 * lpSum(V最低コマ数)
    + C管理者不足 * lpSum(a.V管理者不足)
    + C1日2コマ * lpSum(V1日2コマ)) # 目的関数

In [14]:
for _,r in a.iterrows():
    m += r.V必要人数差 >=  (lpSum(V割当[r.name]) - r.必要人数)
    m += r.V必要人数差 >= -(lpSum(V割当[r.name]) - r.必要人数)
    m += lpSum(V割当[r.name,L管理者]) + r.V管理者不足 >= 1

In [15]:
for j,n in enumerate((a.iloc[:,L従業員].sum(0)+1)//2):
    m += lpSum(V割当[:,j]) + V最低コマ数[j] >= n # 希望の半分以上

In [16]:
for _,v in a.groupby('曜日'):
    for j in range(N従業員):
        m += lpSum(V割当[v.index,j]) + V1日2コマ[j] <= 2 # 各曜日で2コマまで

In [17]:
%time m.solve()

CPU times: user 23.3 ms, sys: 7.78 ms, total: 31.1 ms
Wall time: 43 ms


1

In [18]:
R結果 = np.vectorize(value)(V割当).astype(int)
display(R結果)

array([[0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 1, 0, 1, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 1, 1, 0, 1, 0, 0, 0]])

In [19]:
a['結果'] = [''.join(i*j for i,j in zip(r,a.columns)) for r in R結果]
display(a)

,従業員0,従業員1,従業員2,従業員3,従業員4,従業員5,従業員6,従業員7,従業員8,従業員9,曜日,時間帯,必要人数,V必要人数差,V管理者不足,結果
0,True,True,False,True,False,True,False,False,False,False,月,朝,2,v0221,v0242,従業員1従業員5
1,False,True,False,True,False,True,False,True,False,False,月,昼,3,v0222,v0243,従業員3従業員5従業員7
2,False,True,False,True,True,True,False,False,True,False,月,夜,3,v0223,v0244,従業員1従業員3従業員4
3,True,False,False,True,False,True,False,False,False,False,火,朝,2,v0224,v0245,従業員0従業員3
4,False,False,False,True,False,True,False,True,False,False,火,昼,3,v0225,v0246,従業員3従業員5従業員7
5,False,False,False,True,True,True,False,False,True,False,火,夜,3,v0226,v0247,従業員4従業員5従業員8
6,True,True,False,True,False,True,False,False,False,False,水,朝,2,v0227,v0248,従業員0従業員5
7,False,True,False,True,False,True,False,True,False,False,水,昼,3,v0228,v0249,従業員1従業員3従業員5
8,False,True,False,True,True,True,False,False,True,False,水,夜,3,v0229,v0250,従業員3従業員4従業員8
9,True,False,False,True,False,True,False,False,False,True,木,朝,1,v0230,v0251,従業員3


In [20]:
print('目的関数', value(m.objective))
print(a[['曜日','時間帯','結果']])

目的関数 0.0
   曜日 時間帯                結果
0   月   朝          従業員1従業員5
1   月   昼      従業員3従業員5従業員7
2   月   夜      従業員1従業員3従業員4
3   火   朝          従業員0従業員3
4   火   昼      従業員3従業員5従業員7
5   火   夜      従業員4従業員5従業員8
6   水   朝          従業員0従業員5
7   水   昼      従業員1従業員3従業員5
8   水   夜      従業員3従業員4従業員8
9   木   朝              従業員3
10  木   昼          従業員5従業員7
11  木   夜          従業員8従業員9
12  金   朝          従業員1従業員5
13  金   昼      従業員1従業員7従業員9
14  金   夜      従業員5従業員6従業員8
15  土   朝          従業員0従業員3
16  土   昼  従業員2従業員6従業員7従業員9
17  土   夜  従業員3従業員4従業員6従業員9
18  日   朝          従業員0従業員9
19  日   昼  従業員2従業員3従業員6従業員9
20  日   夜  従業員2従業員3従業員4従業員6
